In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from maritime_traffic_network import MaritimeTrafficNetwork
import folium
import warnings
import sys
warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# add paths for modules
sys.path.append('../visualization')
print(sys.path)

# import modules
import visualize

['/Users/janhendrikwebert/maritime_route_prediction/src/models', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python311.zip', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/lib-dynload', '', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/site-packages', '../visualization']


In [3]:
# read data from file
filename = '../../data/processed/202204_points_stavanger_cleaned_meta_200k.parquet'
#filename = '../../data/processed/202204_points_stavanger_cleaned_full.parquet'
gdf = gpd.read_parquet(filename)
gdf.head()

,mmsi,imo_nr,length,lon,lat,sog,cog,true_heading,nav_status,message_nr,bredde,dypgaaende,skipstype,skipsgruppe,fartoynavn,geometry,speed
date_time_utc,,,,,,,,,,,,,,,,,
2022-04-01 06:30:21,209989000_0,9235505,90,4.62360,59.5881,10.0,167.2,174,0,1,13.6,6.36,General Cargo Ship,Last,LISA,POINT (4.62360 59.58810),4.473722
2022-04-01 06:30:31,209989000_0,9235505,90,4.62367,59.5877,9.7,179.6,174,0,1,13.6,6.36,General Cargo Ship,Last,LISA,POINT (4.62367 59.58770),4.473722
2022-04-01 06:30:40,209989000_0,9235505,90,4.62375,59.5873,9.9,173.0,174,0,1,13.6,6.36,General Cargo Ship,Last,LISA,POINT (4.62375 59.58730),4.976744
2022-04-01 06:30:50,209989000_0,9235505,90,4.62384,59.5868,9.8,174.7,174,0,1,13.6,6.36,General Cargo Ship,Last,LISA,POINT (4.62384 59.58680),5.593419
2022-04-01 06:31:10,209989000_0,9235505,90,4.62402,59.5859,9.7,177.4,174,0,1,13.6,6.36,General Cargo Ship,Last,LISA,POINT (4.62402 59.58590),5.038954


In [4]:
# initialize maritime traffic network
network = MaritimeTrafficNetwork(gdf)
network.get_trajectories_info()

AIS messages: 192346
Trajectories: 249


In [5]:
# calculate significant turning points using Douglas Peucker algorithm
tolerance = 0.002  # DP tolerance parameter
network.calc_significant_points_DP(tolerance)

Calculating significant turning points with Douglas Peucker algorithm (tolerance = 0.002) ...
Number of significant points detected: 2924 (1.52% of AIS messages)
Time elapsed: 0.12 minutes
Adding course over ground before and after each turn ...
Done. Time elapsed: 0.03 minutes


In [6]:
simplified_trajectories = network.significant_points_trajectory

In [7]:
traj_df = network.significant_points
traj_df.reset_index(inplace=True)
mmsi = '257231000_8'
traj = traj_df[(traj_df.mmsi == mmsi)]
traj[['mmsi', 'cog_before', 'cog_after']]

,mmsi,cog_before,cog_after
939,257231000_8,95.787786,95.787786
940,257231000_8,95.787786,75.491415
941,257231000_8,75.491415,92.309732
942,257231000_8,92.309732,131.220781
943,257231000_8,131.220781,134.172108
944,257231000_8,134.172108,126.775712
945,257231000_8,126.775712,89.229464
946,257231000_8,89.229464,89.229464


In [8]:
# plotting
detailed_plot = True
if detailed_plot:
    columns = ['geometry', 'mmsi', 'skipsgruppe', 'fartoynavn', 'cog_before', 'cog_after']  # columns to be plotted
    # plot simplified trajectories
    map = traj[columns].explore(column='skipsgruppe', name='Simplified trajectories', 
                                                   style_kwds={'weight':2}, cmap='jet')
map